<a href="https://colab.research.google.com/github/saddam2973/GEN-AI/blob/Assignments/RAG_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install --upgrade pip
!pip install sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic




In [55]:
!pip install unstructured openpyxl pandas
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1


E: Package 'libfluidsynth1' has no installation candidate


In [56]:
import os
from getpass import getpass

if not os.getenv("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")

In [57]:
!pip install unstructured python-pptx
!pip install unstructured openpyxl pandas
!pip install msoffcrypto-tool
!pip install unstructured python-docx

!pip install unstructured





In [69]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
import os


loader = UnstructuredExcelLoader("/content/2_Functions_Intro.xlsx")
docs = loader.load()





splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"Loaded {len(docs)} docs, split into {len(chunks)} chunks")
# inspect one
print(chunks[0].page_content[:400])

Loaded 1 docs, split into 15 chunks
Job Title Experience (Years) Annual Salary ($USD) Bonus Max ($USD) Experience (=) Total Salary (+) Bonus Rate (/) Confirm Total Salary Does Total Salary = Confirmed Salary? Is Bonus > Annual Salary? Meets Experience Meets Salary Meets Both (1 or 0) Meets Both - Formula Only Meets Both - AND() Job


In [59]:
print("Creating embeddings... this may take a moment.")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Creating embeddings... this may take a moment.


In [60]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [61]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [62]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-tiny")

In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")

In [64]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [65]:
rag_chain = create_retrieval_chain(retriever, document_chain)

In [66]:
os.environ["MISTRAL_API_KEY"]

'QIS9Um5UE38FRDNlRP8VZcZkr8juPpCe'

In [68]:
user_query = "What is Facial Expression ?"
response = rag_chain.invoke({"input": user_query})

print("\n--- RESPONSE ---")
print(response["answer"])


--- RESPONSE ---
Facial Expression, according to the provided context, refers to one of the most powerful forms of non-verbal communication used by humans to express emotions such as happiness, sadness, anger, fear, and surprise.
